In [2]:
import pandas as pd

In [6]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [7]:
# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [3]:
df = pd.read_csv("transcript_new_sentences.csv")
df

,Filename,Sentences_new
0,Wizard-Lesson 1-01.wma,"Conversation, Lesson 1\n-separator-\nDid the d..."
1,Wizard-Lesson 10-10.wma,"Conversation, Lesson #10\n-separator-\nIs he a..."
2,Wizard-Lesson 11-11.wma,"Conversation, Lesson 11\n-separator-\nWhere do..."
3,Wizard-Lesson 12-12.wma,"Conversation, Lesson 12 \n-separator- \nShe ..."
4,Wizard-Lesson 13-13.wma,"Conversation, Lesson 13\n-separator-\nHe assig..."
5,Wizard-Lesson 14-14.wma,Conversation Lesson 14\n-separator-\nShe deplo...
6,Wizard-Lesson 14-14.wma,Conversation Lesson 14\n-separator-\nHe deplo...
7,Wizard-Lesson 15-15.wma,"Conversation, Lesson 15 \n-separator- \nTher..."
8,Wizard-Lesson 16-16.wma,Conversation Lesson 16\n-separator-\nIs there ...
9,Wizard-Lesson 17-17.wma,"Conversation, Lesson 17\n-separator-\nAre ther..."


In [59]:
import os

In [78]:
for i, row in df.iterrows():
    lesson_name = row["Filename"].split(".")[0]
    print( i, lesson_name )

    folder = f"new_audio/{lesson_name}"
    if not os.path.exists(folder):
        os.makedirs(folder)

    for j, line in enumerate( row["Sentences_new"].replace("-separator-", "").split("\n")[1:] ):
        if line != "":
            print( line )
            

            inputs = processor(text=line, return_tensors="pt")
            speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

            sf.write(f"{folder}/{j:03d}.wav", speech.numpy(), samplerate=16000)

    #     if j >0:
    #         break

    # if i == 0:
    #     break

0 Wizard-Lesson 1-01
Did the developer deploy the new feature to the server?
Yes, he did. He deployed the new feature to the server.
No, he didn't. He didn't deploy the new feature to the server.
Did the project manager update the task list on the board?
No, she didn't. She didn't update the task list on the board.
Did you fetch the data from the database?
Yes, I did. I fetched the data from the database.
Did your colleague complete the coding tasks ahead of schedule?
Yes, she did. She completed the coding tasks ahead of schedule.
No, she didn't. She didn't complete the coding tasks ahead of schedule.
Did they implement a new algorithm in the machine learning model?
No, they didn't. They didn't implement a new algorithm in the machine learning model.
Is your fellow programmer debugging the code in the office?
Yes, he is. He is debugging the code in the office.
No, he isn't. He isn't debugging the code in the office.
Are the interns testing the software in the virtual environment?
Yes, 